In [1]:
# google 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 패키지 로드
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from collections import defaultdict
import os, random

from scipy import sparse
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.nn.init import normal_
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F


In [3]:
# 하이퍼파라미터 
class cfg: 
    gpu_idx = 0
    device = torch.device("cuda:{}".format(gpu_idx) if torch.cuda.is_available() else "cpu")
    top_k = 25 # 상위 25개만 선택 
    seed = 42 # seed 
    
    neg_ratio = 150

In [4]:
# 시드 고정 
def seed_everything(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)
    
seed_everything(cfg.seed)

In [5]:
# 모델 
class NeuMF(nn.Module):
    def __init__(self, cfg):
        super(NeuMF, self).__init__()
        self.n_users = cfg.n_users # 유저 수
        self.n_items = cfg.n_items # 아이템 수 
        self.emb_dim = cfg.emb_dim # embedding 차원 수
        self.layer_dim = cfg.layer_dim # layer 차원 수 (2, 2),
        self.n_continuous_feats = cfg.n_continuous_feats # 1 
        self.n_genres = cfg.n_genres # 장르 수 (29)
        self.dropout = cfg.dropout # dropout 비율
        self.build_graph() 

    def build_graph(self):
        self.user_embedding_mf = nn.Embedding(num_embeddings=self.n_users, embedding_dim=self.emb_dim) # user embedding
        self.item_embedding_mf = nn.Embedding(num_embeddings=self.n_items, embedding_dim=self.emb_dim) # item embedding
        
        self.user_embedding_mlp = nn.Embedding(num_embeddings=self.n_users, embedding_dim=self.emb_dim) # user embedding
        self.item_embedding_mlp = nn.Embedding(num_embeddings=self.n_items, embedding_dim=self.emb_dim) # item embedding
                
        self.genre_embeddig = nn.Embedding(num_embeddings=self.n_genres, embedding_dim=self.n_genres//2) # 14로 embedding (n_geners = 29)
        
        self.mlp_layers = nn.Sequential(
            nn.Linear(2*self.emb_dim + self.n_genres//2 + self.n_continuous_feats, self.layer_dim), 
            nn.ReLU(), 
            nn.Dropout(p=self.dropout), 
            nn.Linear(self.layer_dim, self.layer_dim//2), 
            nn.ReLU(), 
            nn.Dropout(p=self.dropout)
        )
        self.affine_output = nn.Linear(self.layer_dim//2 + self.emb_dim, 1)
        self.apply(self._init_weights)
        

    def _init_weights(self, module):
        if isinstance(module, nn.Embedding):
            normal_(module.weight.data, mean=0.0, std=0.01)
        elif isinstance(module, nn.Linear):
            normal_(module.weight.data, 0, 0.01)
            if module.bias is not None:
                module.bias.data.fill_(0.0)
    
    def forward(self, user_indices, item_indices, feats):
        user_embedding_mf = self.user_embedding_mf(user_indices)
        item_embedding_mf = self.item_embedding_mf(item_indices)
        mf_output = torch.mul(user_embedding_mf, item_embedding_mf) # element wise 
        
        user_embedding_mlp = self.user_embedding_mlp(user_indices)
        item_embedding_mlp = self.item_embedding_mlp(item_indices)
        genre_embedding_mlp = self.genre_embeddig(feats[1]) # 장르 
        input_feature = torch.cat((user_embedding_mlp, item_embedding_mlp, genre_embedding_mlp, feats[0].unsqueeze(1)), -1) # feat[0] > age
        mlp_output = self.mlp_layers(input_feature)
        
        output = torch.cat([mlp_output, mf_output], dim=-1)
        output = self.affine_output(output).squeeze(-1)
        return output

In [6]:
def make_UIdataset(train, neg_ratio):
    UIdataset = {}
    for user_id, items_by_user in enumerate(train): # 0 / [0,0,0,0,.....0,0,0,0]
        UIdataset[user_id] = []
        # positive 샘플 계산 
        pos_item_ids = np.where(items_by_user > 0.5)[0] # 1인 곳의 인덱스만 표현
        num_pos_samples = len(pos_item_ids) # 1이 몇개인지

        # negative 샘플 계산 (random negative sampling) 
        num_neg_samples = neg_ratio * num_pos_samples # 1의 갯수에 따라 달라짐
        neg_items = np.where(items_by_user < 0.5)[0] # 0인 곳의 인덱스 표현

        neg_item_ids = np.random.choice(neg_items, min(num_neg_samples, len(neg_items)), replace=False) # 최소값 갯수
        UIdataset[user_id].append(np.concatenate([pos_item_ids, neg_item_ids])) # 본것, 안본것 idx concat 
        
        # feature 추출 
        features = []

        for item_id in np.concatenate([pos_item_ids, neg_item_ids]): #[1,2,3,4,5,6,7] [3,4,6,121,4,35,35] // [1,1,1,1,1,1,1,1]
            features.append(user_features['age'][user_id]) #age feature > content니까 
        UIdataset[user_id].append(np.array(features)) # [indx][age feature]
        
        features = []
        for item_id in np.concatenate([pos_item_ids, neg_item_ids]): 
            features.append(item_features['genre_mid'][item_id])
        UIdataset[user_id].append(np.array(features))
        
        # label 저장  
        pos_labels = np.ones(len(pos_item_ids))#[1,2,3,4]
        neg_labels = np.zeros(len(neg_item_ids))#[6,4,2,4]
        UIdataset[user_id].append(np.concatenate([pos_labels, neg_labels]))#[1,1,1,1,0,0,0,0]

    return UIdataset

In [7]:
def make_batchdata(user_indices, batch_idx, batch_size):
    batch_user_indices = user_indices[batch_idx*batch_size : (batch_idx+1)*batch_size]
    batch_user_ids = []
    batch_item_ids = []
    batch_feat0 = []
    batch_feat1 = []
    batch_labels = []
    for user_id in batch_user_indices:
        item_ids = UIdataset[user_id][0] # index
        feat0 = UIdataset[user_id][1] # age
        feat1 = UIdataset[user_id][2] # 장르
        labels = UIdataset[user_id][3] # 정답 
        user_ids = np.full(len(item_ids), user_id) # [5,5,5,,5,5]
        batch_user_ids.extend(user_ids.tolist())
        batch_item_ids.extend(item_ids.tolist())

        batch_feat0.extend(feat0.tolist())
        batch_feat1.extend(feat1.tolist())
        batch_labels.extend(labels.tolist())
    return batch_user_ids, batch_item_ids, batch_feat0, batch_feat1, batch_labels

def update_avg(curr_avg, val, idx):
    return (curr_avg * idx + val) / (idx + 1)

#### 학습 및 검증 코드 생성

In [8]:
def train_epoch(cfg, model, optimizer, criterion): 
    model.train()
    curr_loss_avg = 0.0

    user_indices = np.arange(cfg.n_users)
    np.random.RandomState(cfg.epoch).shuffle(user_indices)
    batch_num = int(len(user_indices) / cfg.batch_size) + 1
    bar = tqdm(range(batch_num), leave=False)
    
    for step, batch_idx in enumerate(bar):
        user_ids, item_ids, feat0, feat1, labels = make_batchdata(user_indices, batch_idx, cfg.batch_size)
        # 배치 사용자 단위로 학습
        user_ids = torch.LongTensor(user_ids).to(cfg.device)
        item_ids = torch.LongTensor(item_ids).to(cfg.device)
        feat0 = torch.FloatTensor(feat0).to(cfg.device)
        feat1 = torch.LongTensor(feat1).to(cfg.device)
        labels = torch.FloatTensor(labels).to(cfg.device)
        labels = labels.view(-1, 1)

        # grad 초기화
        optimizer.zero_grad()

        # 모델 forward
        output = model.forward(user_ids, item_ids, [feat0, feat1])
        output = output.view(-1, 1)

        loss = criterion(output, labels)

        # 역전파
        loss.backward()

        # 최적화
        optimizer.step()    
        if torch.isnan(loss):
            print('Loss NAN. Train finish.')
            break
        curr_loss_avg = update_avg(curr_loss_avg, loss, step)
        
        msg = f"epoch: {cfg.epoch}, "
        msg += f"loss: {curr_loss_avg.item():.5f}, "
        msg += f"lr: {optimizer.param_groups[0]['lr']:.6f}"
        bar.set_description(msg)
    rets = {'losses': np.around(curr_loss_avg.item(), 5)}
    return rets

In [9]:
def recallk(actual, predicted, k = 25):
    set_actual = set(actual)
    recall_k = len(set_actual & set(predicted[:k])) / min(k, len(set_actual))
    return recall_k

def unique(sequence):
    # preserves order
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

def ndcgk(actual, predicted, k = 25):
    set_actual = set(actual)
    idcg = sum([1.0 / np.log(i + 2) for i in range(min(k, len(set_actual)))])
    dcg = 0.0
    unique_predicted = unique(predicted[:k])
    for i, r in enumerate(unique_predicted):
        if r in set_actual:
            dcg += 1.0 / np.log(i + 2)
    ndcg_k = dcg / idcg
    return ndcg_k

def evaluation(gt, pred):
    gt = gt.groupby('profile_id')['album_id'].unique().to_frame().reset_index()
    gt.columns = ['profile_id', 'actual_list']

    evaluated_data = pd.merge(pred, gt, how = 'left', on = 'profile_id')

    evaluated_data['Recall@25'] = evaluated_data.apply(lambda x: recallk(x.actual_list, x.predicted_list), axis=1)
    evaluated_data['NDCG@25'] = evaluated_data.apply(lambda x: ndcgk(x.actual_list, x.predicted_list), axis=1)

    recall = evaluated_data['Recall@25'].mean()
    ndcg = evaluated_data['NDCG@25'] .mean()
    coverage = (evaluated_data['predicted_list'].apply(lambda x: x[:cfg.top_k]).explode().nunique())/meta_df.index.nunique()

    score = 0.75*recall + 0.25*ndcg
    rets = {"recall" :recall, 
            "ndcg" :ndcg, 
            "coverage" :coverage, 
            "score" :score}
    return rets

In [10]:
def valid_epoch(cfg, model, data, mode='valid'):
    pred_list = []
    model.eval()
    
    query_user_ids = data['profile_id'].unique() # 추론할 모든 user array 집합
    full_item_ids = np.array([c for c in range(cfg.n_items)]) # 추론할 모든 item array 집합 
    full_item_ids_feat1 = [item_features['genre_mid'][c] for c in full_item_ids]
    for user_id in query_user_ids:
        with torch.no_grad():
            user_ids = np.full(cfg.n_items, user_id)
            
            user_ids = torch.LongTensor(user_ids).to(cfg.device)
            item_ids = torch.LongTensor(full_item_ids).to(cfg.device)
            
            feat0 = np.full(cfg.n_items, user_features['age'][user_id])
            feat0 = torch.FloatTensor(feat0).to(cfg.device)
            feat1 = torch.LongTensor(full_item_ids_feat1).to(cfg.device)
            
            eval_output = model.forward(user_ids, item_ids, [feat0, feat1]).detach().cpu().numpy()
            pred_u_score = eval_output.reshape(-1)   
        
        pred_u_idx = np.argsort(pred_u_score)[::-1]
        pred_u = full_item_ids[pred_u_idx]
        pred_list.append(list(pred_u[:cfg.top_k]))
        
    pred = pd.DataFrame()
    pred['profile_id'] = query_user_ids
    pred['predicted_list'] = pred_list
    
    # 모델 성능 확인 
    if mode == 'valid':
        rets = evaluation(data, pred)
        return rets, pred
    return pred

In [11]:
# 경로 설정
data_path = '/content/drive/MyDrive/데이콘/[대회]LG추천/제출/data'
saved_path = '/content/drive/MyDrive/데이콘/[대회]LG추천/제출/model'
output_path = '/content/drive/MyDrive/데이콘/[대회]LG추천/제출/submission'

In [12]:
# 데이터 불러오기 
history_df = pd.read_csv(os.path.join(data_path, 'history_data.csv'), encoding='utf-8')
profile_df = pd.read_csv(os.path.join(data_path, 'profile_data.csv'), encoding='utf-8')
meta_df = pd.read_csv(os.path.join(data_path, 'meta_data.csv'), encoding='utf-8')

In [13]:
# 처음 시청한 기록을 기준으로 하기 위해 ss_id를 기준으로 삭제
data = history_df[['profile_id', 'ss_id', 'album_id']].drop_duplicates(subset=['profile_id', 'album_id', 'ss_id']).sort_values(by = ['profile_id', 'ss_id']).reset_index(drop = True)
data['rating'] = 1

cfg.n_users = data.profile_id.max()+1
cfg.n_items = data.album_id.max()+1

## 모델 학습

### 하이퍼파라미터 설정 & 최적화 기법 설정

In [14]:
# 하이퍼 파라미터 설정 
cfg.batch_size = 256
cfg.emb_dim = 128
cfg.layer_dim = 128
cfg.dropout = 0.00
cfg.epochs = 20
cfg.learning_rate = 0.001
cfg.reg_lambda = 0
cfg.check_epoch = 1

cfg.n_genres = meta_df['genre_mid'].nunique()
cfg.n_continuous_feats = 1 

### 학습 및 검증 데이터 생성 (K-fold)

In [15]:
from sklearn.model_selection import KFold


kf = KFold(n_splits=5,shuffle = True, random_state= 42)
count = 0
for  train_index, test_index in kf.split(data):
  count += 1
  print("TRAIN:", train_index, "TEST:", test_index)

  train = data.loc[train_index]
  valid = data.loc[test_index]

  # Matrix 형태로 변환 
  train = train.to_numpy()
  matrix = sparse.lil_matrix((cfg.n_users, cfg.n_items))
  print(matrix.shape)
  for (p, _, i, r) in (train): # profile_id  / album_id 
      matrix[p, i] = r
      
  train = sparse.csr_matrix(matrix)
  train = train.toarray()
  # train을 인접 행렬로 생성

  # 유저 특징 정보 추출 
  temp_profile_df = profile_df.set_index('profile_id')
  user_features = temp_profile_df[['age']].to_dict()
  # user_features는 나이를 dictionary 형태로 생성

  # 아이템 특징 정보 추출 
  temp_meta_df = meta_df.set_index('album_id')

  # 범주형 데이터를 수치형 데이터로 변경 
  le = LabelEncoder()
  temp_meta_df['genre_mid'] = le.fit_transform(temp_meta_df['genre_mid'])
  item_features = temp_meta_df[['genre_mid']].to_dict()
  # content features는 genre mid만 사용
  
  ######################################
  # data set 생성
  UIdataset = make_UIdataset(train, neg_ratio=cfg.neg_ratio)

  ########################### 모델 생성
  model = NeuMF(cfg).to(cfg.device)
  optimizer = torch.optim.Adam(model.parameters(), lr=cfg.learning_rate, weight_decay=cfg.reg_lambda)
  criterion = torch.nn.BCEWithLogitsLoss(reduction='sum')

  ########################### 학습 진행
  total_logs = defaultdict(list)
  best_scores  = 0
  for epoch in range(cfg.epochs+1):
      cfg.epoch = epoch
      train_results = train_epoch(cfg, model, optimizer, criterion)
      
      # cfg.check_epoch 번의 epoch 마다 성능 확인 
      if epoch % cfg.check_epoch == 0: 
          valid_results, _ = valid_epoch(cfg, model, valid)

          logs = {
              'Train Loss': train_results['losses'],
              f'Valid Recall@{cfg.top_k}': valid_results['recall'],
              f'Valid NDCG@{cfg.top_k}': valid_results['ndcg'],
              'Valid Coverage': valid_results['coverage'],
              'Valid Score': valid_results['score'],
              }

          # 검증 성능 확인 
          for key, value in logs.items():
              total_logs[key].append(value)

          if epoch == 0:
              print("Epoch", end=",")
              print(",".join(logs.keys()))

          print(f"{epoch:02d}  ", end="")
          print("  ".join([f"{v:0.6f}" for v in logs.values()]))
          
          # 가장 성능이 좋은 가중치 파일을 저장 
          if best_scores <= valid_results['score']: 
              best_scores = valid_results['score']
              torch.save(model.state_dict(), os.path.join(saved_path, f'model(best_scores)_{count}.pth'))

print(f"finish!!!!!!")

TRAIN: [     0      1      3 ... 670765 670766 670768] TEST: [     2      6      7 ... 670759 670762 670767]
(33033, 25917)


  0%|          | 0/130 [00:00<?, ?it/s]

Epoch,Train Loss,Valid Recall@25,Valid NDCG@25,Valid Coverage,Valid Score
00  61992.011720  0.136250  0.086246  0.002512  0.123749


  0%|          | 0/130 [00:00<?, ?it/s]

01  10965.261720  0.153981  0.111656  0.002136  0.143400


...

18  2282.571290  0.435011  0.301962  0.273673  0.401749


  0%|          | 0/130 [00:00<?, ?it/s]

19  2071.499510  0.434778  0.301509  0.283390  0.401461


  0%|          | 0/130 [00:00<?, ?it/s]

20  1886.264040  0.433629  0.300597  0.291653  0.400371
finish!!!!!!
